In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt  # 시각화를 위해 추가

class MaskIoUCalculator:
    def __init__(self, gt_masks_dir, pred_masks_dir, output_dir, size=(512, 512)):
        self.gt_masks_dir = gt_masks_dir
        self.pred_masks_dir = pred_masks_dir
        self.output_dir = output_dir
        self.size = size

        # 결과 저장을 위한 디렉토리 생성
        self.results_dir = os.path.join(output_dir, "comparison_results")
        os.makedirs(self.results_dir, exist_ok=True)

        # 결과를 저장할 파일
        self.results_file = os.path.join(output_dir, "iou_results.txt")

    def load_and_preprocess_mask(self, mask_path):
    
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Failed to load mask: {mask_path}")
            
        # 512x512로 리사이즈
        mask = cv2.resize(mask, self.size, interpolation=cv2.INTER_NEAREST)
        
        # 마스크의 최대값과 최소값 출력
        print(f"Mask {mask_path} - min: {mask.min()}, max: {mask.max()}")
        
        # 마스크 픽셀 값 스케일링 또는 비제로 값 처리
        if mask.max() <= 1:
            mask = mask * 255  # 최대값이 1 이하인 경우
        elif mask.max() <= 127:
            # 픽셀 값 스케일링
            mask = cv2.normalize(mask, None, 0, 255, cv2.NORM_MINMAX)
        else:
            # 비제로 픽셀을 255로 설정
            mask[mask > 0] = 255
        
        # 이진화 (0 또는 255로)
        _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        
        # 마스크의 합계를 출력하여 객체가 있는지 확인
        mask_sum = np.sum(mask)
        print(f"Mask sum for {mask_path}: {mask_sum}")
        
        return mask


    def calculate_iou(self, gt_mask, pred_mask):
        """IoU 계산"""
        # 마스크를 불리언 형태로 변환
        gt_mask_bool = gt_mask > 0
        pred_mask_bool = pred_mask > 0

        # 논리 연산 수행
        intersection = np.logical_and(gt_mask_bool, pred_mask_bool).sum()
        union = np.logical_or(gt_mask_bool, pred_mask_bool).sum()
        if union == 0:
            return 0
        return intersection / union

    def calculate_giou(self, gt_mask, pred_mask):
        """GIoU 계산"""
        # 마스크를 불리언으로 변환
        gt_mask = gt_mask > 0
        pred_mask = pred_mask > 0

        # Bounding box 좌표 추출
        def get_bbox(mask):
            y, x = np.nonzero(mask)
            if len(y) == 0 or len(x) == 0:
                return [0, 0, 0, 0]
            return [x.min(), y.min(), x.max(), y.max()]

        gt_bbox = get_bbox(gt_mask)
        pred_bbox = get_bbox(pred_mask)

        # Intersection box
        ix1 = max(gt_bbox[0], pred_bbox[0])
        iy1 = max(gt_bbox[1], pred_bbox[1])
        ix2 = min(gt_bbox[2], pred_bbox[2])
        iy2 = min(gt_bbox[3], pred_bbox[3])

        # Intersection area
        intersection = max(0, ix2 - ix1) * max(0, iy2 - iy1)

        # Union area
        gt_area = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])
        pred_area = (pred_bbox[2] - pred_bbox[0]) * (pred_bbox[3] - pred_bbox[1])
        union = gt_area + pred_area - intersection

        # Convex hull area (smallest enclosing box)
        cx1 = min(gt_bbox[0], pred_bbox[0])
        cy1 = min(gt_bbox[1], pred_bbox[1])
        cx2 = max(gt_bbox[2], pred_bbox[2])
        cy2 = max(gt_bbox[3], pred_bbox[3])
        convex_area = max(0, cx2 - cx1) * max(0, cy2 - cy1)

        if union == 0:
            return 0

        iou = intersection / union
        return iou - ((convex_area - union) / convex_area)

    def calculate_ciou(self, gt_mask, pred_mask):
        """CIoU 계산"""
        # 마스크를 불리언으로 변환
        gt_mask = gt_mask > 0
        pred_mask = pred_mask > 0

        # Bounding box 좌표 추출
        def get_bbox(mask):
            y, x = np.nonzero(mask)
            if len(y) == 0 or len(x) == 0:
                return [0, 0, 0, 0]
            return [x.min(), y.min(), x.max(), y.max()]

        gt_bbox = get_bbox(gt_mask)
        pred_bbox = get_bbox(pred_mask)

        # bbox가 유효한지 검사
        if (gt_bbox[3] - gt_bbox[1]) <= 0 or (pred_bbox[3] - pred_bbox[1]) <= 0:
            return 0.0  # 유효하지 않은 bbox인 경우 0 반환

        # 중심점 계산
        gt_center = [(gt_bbox[0] + gt_bbox[2]) / 2, (gt_bbox[1] + gt_bbox[3]) / 2]
        pred_center = [(pred_bbox[0] + pred_bbox[2]) / 2, (pred_bbox[1] + pred_bbox[3]) / 2]

        # 중심점 간의 유클리드 거리
        center_distance = np.sqrt((gt_center[0] - pred_center[0]) ** 2 +
                                  (gt_center[1] - pred_center[1]) ** 2)

        # 대각선 거리 (convex hull)
        cx1 = min(gt_bbox[0], pred_bbox[0])
        cy1 = min(gt_bbox[1], pred_bbox[1])
        cx2 = max(gt_bbox[2], pred_bbox[2])
        cy2 = max(gt_bbox[3], pred_bbox[3])
        convex_diag = np.sqrt((cx2 - cx1) ** 2 + (cy2 - cy1) ** 2)

        # IoU 계산
        intersection = np.logical_and(gt_mask, pred_mask).sum()
        union = np.logical_or(gt_mask, pred_mask).sum()

        if union == 0:
            return 0

        iou = intersection / union

        # epsilon 추가하여 0으로 나누기 방지
        eps = 1e-7
        gt_height = max(gt_bbox[3] - gt_bbox[1], eps)
        gt_width = max(gt_bbox[2] - gt_bbox[0], eps)
        pred_height = max(pred_bbox[3] - pred_bbox[1], eps)
        pred_width = max(pred_bbox[2] - pred_bbox[0], eps)

        gt_aspect_ratio = gt_width / gt_height
        pred_aspect_ratio = pred_width / pred_height

        v = (4 / (np.pi ** 2)) * (np.arctan(pred_aspect_ratio) - np.arctan(gt_aspect_ratio)) ** 2
        alpha = v / (1 - iou + v + eps)  # eps 추가

        if convex_diag < eps:  # 대각선이 너무 작은 경우
            return iou

        ciou = iou - (center_distance ** 2) / (convex_diag ** 2 + eps) - alpha * v

        return ciou

    def visualize_comparison(self, gt_mask, pred_mask, comparison_path):
        """마스크 비교 시각화"""
        # 두 마스크를 RGB로 변환
        gt_vis = np.zeros((self.size[0], self.size[1], 3), dtype=np.uint8)
        pred_vis = np.zeros((self.size[0], self.size[1], 3), dtype=np.uint8)

        gt_vis[gt_mask > 0] = [0, 255, 0]    # 초록색으로 GT 표시
        pred_vis[pred_mask > 0] = [255, 0, 0]  # 빨간색으로 예측 표시

        # 두 마스크를 합치기
        comparison = cv2.addWeighted(gt_vis, 0.5, pred_vis, 0.5, 0)

        # 겹치는 부분을 노란색으로 표시
        overlap = np.logical_and(gt_mask > 0, pred_mask > 0)
        comparison[overlap] = [0, 255, 255]

        cv2.imwrite(comparison_path, comparison)

    def process_masks(self):
        """모든 마스크 쌍에 대해 IoU 계산"""
        results = []

        # LISA prediction 마스크 파일 목록
        pred_files = [f for f in os.listdir(self.pred_masks_dir) if f.endswith('_combined_mask.png')]

        print(f"Processing {len(pred_files)} mask pairs...")

        with open(self.results_file, 'w') as f:
            f.write("Filename,IoU,GIoU,CIoU\n")

            for pred_file in tqdm(pred_files):
                # LISA 출력에서 원본 파일 이름 추출 (예: "000000000139_mask_0.png" -> "000000000139")
                base_name = pred_file.replace('_combined_mask.png', '')
                gt_file = f"{base_name}.png"  # GT 마스크 파일 이름

                gt_path = os.path.join(self.gt_masks_dir, gt_file)
                pred_path = os.path.join(self.pred_masks_dir, pred_file)

                # GT 파일이 존재하는지 확인
                if not os.path.exists(gt_path):
                    print(f"No ground truth found for {pred_file}")
                    continue

                try:
                    # 마스크 로드 및 전처리
                    gt_mask = self.load_and_preprocess_mask(gt_path)
                    pred_mask = self.load_and_preprocess_mask(pred_path)

                    # IoU 계산
                    iou = self.calculate_iou(gt_mask, pred_mask)
                    giou = self.calculate_giou(gt_mask, pred_mask)
                    ciou = self.calculate_ciou(gt_mask, pred_mask)

                    # 결과 저장
                    results.append({
                        'filename': pred_file,
                        'iou': iou,
                        'giou': giou,
                        'ciou': ciou
                    })

                    # 결과를 파일에 쓰기
                    f.write(f"{pred_file},{iou:.4f},{giou:.4f},{ciou:.4f}\n")

                    # 개별 IoU 출력
                    print(f"Processed {pred_file} - IoU: {iou:.4f}, GIoU: {giou:.4f}, CIoU: {ciou:.4f}")

                    # 비교 시각화 저장
                    comparison_path = os.path.join(self.results_dir, f"{base_name}_comparison.png")
                    self.visualize_comparison(gt_mask, pred_mask, comparison_path)

                except Exception as e:
                    print(f"Error processing {pred_file}: {str(e)}")
                    continue

        if not results:
            print("No valid mask pairs were processed!")
            return

        # 전체 통계 계산
        avg_iou = np.mean([r['iou'] for r in results])
        avg_giou = np.mean([r['giou'] for r in results])
        avg_ciou = np.mean([r['ciou'] for r in results])

        # 요약 정보 추가
        with open(self.results_file, 'a') as f:
            f.write(f"\nSummary Statistics:\n")
            f.write(f"Average IoU: {avg_iou:.4f}\n")
            f.write(f"Average GIoU: {avg_giou:.4f}\n")
            f.write(f"Average CIoU: {avg_ciou:.4f}\n")
            f.write(f"Total processed pairs: {len(results)}\n")

        print(f"\nResults saved to {self.results_file}")
        print(f"Visualizations saved to {self.results_dir}")
        print(f"\nAverage IoU: {avg_iou:.4f}")
        print(f"Average GIoU: {avg_giou:.4f}")
        print(f"Average CIoU: {avg_ciou:.4f}")


def main():
    # 설정
    gt_masks_dir = "/home/joongwon00/coco_dataset/person_masks"
    pred_masks_dir = "/home/joongwon00/qwen_grid_output/combined/masks"
    output_dir = "./iou_evaluation"

    # 계산기 초기화 및 실행
    calculator = MaskIoUCalculator(gt_masks_dir, pred_masks_dir, output_dir)
    calculator.process_masks()

if __name__ == "__main__":
    main()


Processing 41 mask pairs...


 24%|██▍       | 10/41 [00:00<00:00, 96.50it/s]

Mask /home/joongwon00/coco_dataset/person_masks/000000516143.png - min: 0, max: 255
Mask sum for /home/joongwon00/coco_dataset/person_masks/000000516143.png: 144585
Mask /home/joongwon00/qwen_grid_output/combined/masks/000000516143_combined_mask.png - min: 0, max: 255
Mask sum for /home/joongwon00/qwen_grid_output/combined/masks/000000516143_combined_mask.png: 136425
Processed 000000516143_combined_mask.png - IoU: 0.8428, GIoU: 0.8751, CIoU: 0.8426
Mask /home/joongwon00/coco_dataset/person_masks/000000000139.png - min: 0, max: 255
Mask sum for /home/joongwon00/coco_dataset/person_masks/000000000139.png: 830025
Mask /home/joongwon00/qwen_grid_output/combined/masks/000000000139_combined_mask.png - min: 0, max: 255
Mask sum for /home/joongwon00/qwen_grid_output/combined/masks/000000000139_combined_mask.png: 90015
Processed 000000000139_combined_mask.png - IoU: 0.0973, GIoU: 0.0989, CIoU: 0.0566
Mask /home/joongwon00/coco_dataset/person_masks/000000434204.png - min: 0, max: 255
Mask sum fo

 49%|████▉     | 20/41 [00:00<00:00, 97.53it/s]

Mask /home/joongwon00/coco_dataset/person_masks/000000434230.png - min: 0, max: 255
Mask sum for /home/joongwon00/coco_dataset/person_masks/000000434230.png: 1947690
Mask /home/joongwon00/qwen_grid_output/combined/masks/000000434230_combined_mask.png - min: 0, max: 255
Mask sum for /home/joongwon00/qwen_grid_output/combined/masks/000000434230_combined_mask.png: 1110270
Processed 000000434230_combined_mask.png - IoU: 0.5562, GIoU: 0.5262, CIoU: 0.5555
Mask /home/joongwon00/coco_dataset/person_masks/000000417911.png - min: 0, max: 255
Mask sum for /home/joongwon00/coco_dataset/person_masks/000000417911.png: 3074535
Mask /home/joongwon00/qwen_grid_output/combined/masks/000000417911_combined_mask.png - min: 0, max: 255
Mask sum for /home/joongwon00/qwen_grid_output/combined/masks/000000417911_combined_mask.png: 2270010


 78%|███████▊  | 32/41 [00:00<00:00, 102.84it/s]

Processed 000000417911_combined_mask.png - IoU: 0.6989, GIoU: 0.9186, CIoU: 0.6985
Mask /home/joongwon00/coco_dataset/person_masks/000000221291.png - min: 0, max: 255
Mask sum for /home/joongwon00/coco_dataset/person_masks/000000221291.png: 5049765
Mask /home/joongwon00/qwen_grid_output/combined/masks/000000221291_combined_mask.png - min: 0, max: 255
Mask sum for /home/joongwon00/qwen_grid_output/combined/masks/000000221291_combined_mask.png: 1675350
Processed 000000221291_combined_mask.png - IoU: 0.3225, GIoU: 0.5852, CIoU: 0.3040
Mask /home/joongwon00/coco_dataset/person_masks/000000032817.png - min: 0, max: 255
Mask sum for /home/joongwon00/coco_dataset/person_masks/000000032817.png: 12796410
Mask /home/joongwon00/qwen_grid_output/combined/masks/000000032817_combined_mask.png - min: 0, max: 255
Mask sum for /home/joongwon00/qwen_grid_output/combined/masks/000000032817_combined_mask.png: 10293330
Processed 000000032817_combined_mask.png - IoU: 0.7404, GIoU: 0.9507, CIoU: 0.7403
Mask 

100%|██████████| 41/41 [00:00<00:00, 102.12it/s]


Results saved to ./iou_evaluation/iou_results.txt
Visualizations saved to ./iou_evaluation/comparison_results

Average IoU: 0.6033
Average GIoU: 0.6874
Average CIoU: 0.5812
